In [1]:
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from IPython.display import display, Markdown
from sklearn.linear_model import Ridge
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import sys

In [2]:
# Set float format for display (e.g., 2 decimal places)
pd.options.display.float_format = '{:.2f}'.format

# Suppress only the specific FutureWarning from pandas
warnings.filterwarnings("ignore")

In [3]:
#using DataCleaning Module

sys.path.append('/kaggle/input/modules/pyfiles')
from datacleaning import DataCleaning

In [4]:
df = pd.read_csv("/kaggle/input/london-house-price-prediction-advanced-techniques/train.csv")

# Use the class
#cleaner = DataCleaning(data)
#cleaner.show_info()
#cleaner.drop_duplicates()
#cleaner.fill_missing(strategy='median', columns=['bathrooms', 'bedrooms', 'livingRooms', 'floorAreaSqM'])
#cleaner.fill_missing(strategy='mode', columns=['tenure', 'propertyType', 'currentEnergyRating'])
#cleaner.remove_outliers(columns=['price','floorAreaSqM', 'bathrooms', 'bedrooms', 'livingRooms']) 
#cleaner.drop_duplicates()
#cleaner.show_info()

# Get the cleaned data
#df = cleaner.get_clean_data()
df["price"] = df["price"]*(10**-6)   
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
display(df.describe())
display(df.info())

,ID,latitude,longitude,bathrooms,bedrooms,floorAreaSqM,livingRooms,sale_month,sale_year,price
count,266325.00,266325.00,266325.00,217846.00,241482.00,252519.00,229285.00,266325.00,266325.00,266325.00
mean,133162.00,51.51,-0.10,1.44,2.50,99.62,1.29,6.81,2012.21,0.62
std,76881.55,0.06,0.09,0.72,1.17,56.45,0.58,3.36,9.14,1.27
min,0.00,51.39,-0.35,1.00,1.00,10.00,1.00,1.00,1995.00,0.01
25%,66581.00,51.47,-0.16,1.00,2.00,63.00,1.00,4.00,2004.00,0.22
50%,133162.00,51.51,-0.11,1.00,2.00,85.00,1.00,7.00,2014.00,0.39
75%,199743.00,51.55,-0.05,2.00,3.00,118.00,1.00,10.00,2021.00,0.65
max,266324.00,51.67,0.14,9.00,9.00,500.00,9.00,12.00,2023.00,100.00


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266325 entries, 0 to 266324
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   266325 non-null  int64  
 1   fullAddress          266325 non-null  object 
 2   postcode             266325 non-null  object 
 3   country              266325 non-null  object 
 4   outcode              266325 non-null  object 
 5   latitude             266325 non-null  float64
 6   longitude            266325 non-null  float64
 7   bathrooms            217846 non-null  float64
 8   bedrooms             241482 non-null  float64
 9   floorAreaSqM         252519 non-null  float64
 10  livingRooms          229285 non-null  float64
 11  tenure               260604 non-null  object 
 12  propertyType         265817 non-null  object 
 13  currentEnergyRating  209511 non-null  object 
 14  sale_month           266325 non-null  int64  
 15  sale_year        

None

In [5]:
questions = """
Startegy to (1) wrangle data (1.1 data completeness)(check for each columns):
1. ✅ `fullAddress` - No change required.
2. ✅ To extract incode from `postcode`.
3. ✅ To drop `country` column. 
4. ✅ `Latitude`/`Logitude` - No change required.
5. ✅ To impute `floorAreaSqM` with most frequent values (mode).
6. ✅ To impute `bathrooms`/`bedrooms`/`living_room`/`tenure`/`property_type`/`currentEnergyRating` with most frequent values (mode).
7. ✅ To generate time features from sale_year+sale_month using DeterministicProcess and merge
"""
display(Markdown(questions))


Startegy to (1) wrangle data (1.1 data completeness)(check for each columns):
1. ✅ `fullAddress` - No change required.
2. ✅ To extract incode from `postcode`.
3. ✅ To drop `country` column. 
4. ✅ `Latitude`/`Logitude` - No change required.
5. ✅ To impute `floorAreaSqM` with most frequent values (mode).
6. ✅ To impute `bathrooms`/`bedrooms`/`living_room`/`tenure`/`property_type`/`currentEnergyRating` with most frequent values (mode).
7. ✅ To generate time features from sale_year+sale_month using DeterministicProcess and merge


In [6]:
# To extract incode from `postcode`
df['postcode'] = df['postcode'].apply(lambda x: x.split(" ")[1])

In [7]:
# To impute `floorAreaSqM` with most frequent values (mode). 
def mode_func(x):
    return x.mode().iloc[0] if not x.mode().empty else np.nan
    
temp_df = df.groupby('outcode')['floorAreaSqM'].agg(
    min='min',
    max='max',
    mode=mode_func,
    median='median',
    mean='mean',
    count='count'
).reset_index()

df['floorAreaSqM'].fillna(temp_df['mode'].mode().iloc[0], inplace=True)


In [8]:
# To drop `country` column
df.drop('country', inplace=True, axis = 1)

In [9]:
# To impute `bathrooms`/`bedrooms`/`living_room`/`tenure`/`property_type`/`currentEnergyRating` with most frequent values (mode). (Function already created)

In [10]:
# # 1. Create temporary datetime for feature generation
# df['temp_date'] = pd.to_datetime(
#     df['sale_year'].astype(str) + '-' + df['sale_month'].astype(str)
# )

# # 2. Initialize DeterministicProcess (uses temp_date internally)
# dp = DeterministicProcess(
#     index=df['temp_date'],  # Pass dates as Series
#     constant=True,
#     seasonal=True,
#     order=12,
#     drop=True,
#     additional_terms=[CalendarFourier(freq='QE', order=2)]
# )

# # 3. Generate features for existing data
# time_features = dp.in_sample()
# df = pd.concat([df, time_features], axis=1)

# # 4. Generate features for next 7 months
# last_date = df['temp_date'].max()
# future_dates = pd.date_range(
#     start=last_date + pd.offsets.MonthBegin(1),
#     periods=7,
#     freq='M'
# )
# future_features = dp.out_of_sample(steps=7, dates=future_dates)

# # 5. Cleanup (remove temp column if needed)
# df = df.drop(columns=['temp_date'])

# # 3. Generate features for testing data
# df = pd.concat([df, future_features], axis=1)

In [11]:
# questions = """
# Startegy to (1) wrangle data (1.2 data encoding)(check for each columns):
# 1. ✅ `fullAddress` - No change required.
# 2. ✅ To extract incode from `postcode`.
# 3. ✅ To drop `country` column. 
# 4. ✅ `Latitude`/`Logitude` - No change required.
# 5. ✅ To impute `floorAreaSqM` with most frequent values (mode).
# 6. ✅ To impute `bathrooms`/`bedrooms`/`living_room`/`tenure`/`property_type`/`currentEnergyRating` with most frequent values (mode).
# 7. ✅ To generate time features from sale_year+sale_month using DeterministicProcess and merge
# """
# display(Markdown(questions))